In [1]:
import csv
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import numpy as np


In [83]:


MAIN_STRING = "carouselNav__list elementFont__resetList recipeCarousel__list"  # class for main recipe category page navlist
CAT_STRING = "category-page-list-content category-page-list-content__recipe-card karma-main-column"  # Class for category page div
CARDS_STRING = "card__titleLink manual-link-behavior elementFont__title margin-8-bottom"  # Class for most popular recipe cards


def get_links():
    source = requests.get('https://www.allrecipes.com/recipes/').text
    soup = BeautifulSoup(source, 'lxml')
    recipe_list = soup.find('ul', class_=MAIN_STRING)

    links = []
    pop_recipes = []
    for a in recipe_list.find_all('a', href=True):
        links.append(a['href'])
        for i, _ in enumerate(links):
            recipe_category = requests.get(links[i]).text
            cat_soup = BeautifulSoup(recipe_category, 'lxml')  # category page
            most_popular = cat_soup.find('div', class_=CAT_STRING)
            for a in most_popular.find_all('a', class_=CARDS_STRING, href=True):
                if re.search('https://www.allrecipes.com/recipe/.*', a['href']):
                    pop_recipes.append(a['href'])

    df = pd.DataFrame(pop_recipes, columns=["link"])
    df.to_csv('list.csv', index=False)
    return df

In [80]:
df= get_links()

In [108]:
link_array = df['link'].to_numpy()
#link_array

In [109]:
recipe_info = []
for link in link_array:
    source = requests.get(link).text
    soup = BeautifulSoup(source, 'lxml')

    title = soup.find('h1').text
    description = soup.find('p').text
    rating = soup.find('span',class_="review-star-text visually-hidden").text

    ingredient_list = soup.find('ul',class_ = "ingredients-section")
    ingredients = []
    for ingredient in ingredient_list.find_all('span',class_="ingredients-item-name elementFont__body"):
        ingredients.append(ingredient.text)

    instruction_list = soup.find('ul',class_ = "instructions-section")
    instructions = []
    for instruction in instruction_list.find_all('p'):
        instructions.append(instruction.text)
    
    recipe_info.append([title,description,rating,ingredients,instructions])



In [126]:

a = pd.DataFrame.from_dict(dict(zip(df['link'],recipe_info))).T



In [131]:
len(link_array)

2339

In [130]:
len(recipe_info)

2339

In [139]:
import pickle
def SaveLists(data,filename):
    open_file = open(f'{filename}.pkl',"wb")
    pickle.dump(data, open_file)
    open_file.close()

def LoadLists(file):
    open_file = open(file, "rb")
    loaded_list = pickle.load(open_file)
    open_file.close()
    return loaded_list

In [140]:
SaveLists(recipe_info)



In [142]:
lists = LoadLists('myPickleFile.pkl')

In [144]:
len(lists)

2339